In [ ]:
from IPython import get_ipython # Import IPython interactive shell interface
from IPython.display import display # Import display function for rich outputs

from typing import Tuple # Import Tuple for type hinting
import cv2 # Import OpenCV for computer vision tasks
import numpy as np # Import NumPy for numerical operations
import numpy.typing as npt # Import NumPy typing module for type hints

In [ ]:
class ViewTransformer:
    def __init__(
            self,
            source: npt.NDArray[np.float32],
            target: npt.NDArray[np.float32]
    ) -> None:
        """
        Initialize the ViewTransformer with source and target points.

        Args:
            source (npt.NDArray[np.float32]): Source points for homography calculation.
            target (npt.NDArray[np.float32]): Target points for homography calculation.

        Raises:
            ValueError: If source and target do not have the same shape or if they are
                not 2D coordinates.
        """
        # Check if source and target have the same shape
        if source.shape != target.shape:
            raise ValueError("Source and target must have the same shape.")
        # Check if points are 2D coordinates
        if source.shape[1] != 2:
            raise ValueError("Source and target points must be 2D coordinates.")

        # Convert source and target to float32 type for OpenCV compatibility
        source = source.astype(np.float32)
        target = target.astype(np.float32)

        # Calculate homography matrix from source to target points
        self.m, _ = cv2.findHomography(source, target)
        # Raise error if homography matrix could not be computed
        if self.m is None:
            raise ValueError("Homography matrix could not be calculated.")

    def transform_points(
            self,
            points: npt.NDArray[np.float32]
    ) -> npt.NDArray[np.float32]:
        """
        Transform the given points using the homography matrix.

        Args:
            points (npt.NDArray[np.float32]): Points to be transformed.

        Returns:
            npt.NDArray[np.float32]: Transformed points.

        Raises:
            ValueError: If points are not 2D coordinates.
        """
        # Return empty if input points array is empty
        if points.size == 0:
            return points

        # Check if input points are 2D coordinates
        if points.shape[1] != 2:
            raise ValueError("Points must be 2D coordinates.")

        # Reshape points for perspectiveTransform function (N,1,2) and convert to float32
        reshaped_points = points.reshape(-1, 1, 2).astype(np.float32)
        # Apply homography transformation to points
        transformed_points = cv2.perspectiveTransform(reshaped_points, self.m)
        # Reshape back to (N,2) and return as float32
        return transformed_points.reshape(-1, 2).astype(np.float32)

    def transform_image(
            self,
            image: npt.NDArray[np.uint8],
            resolution_wh: Tuple[int, int]
    ) -> npt.NDArray[np.uint8]:
        """
        Transform the given image using the homography matrix.

        Args:
            image (npt.NDArray[np.uint8]): Image to be transformed.
            resolution_wh (Tuple[int, int]): Width and height of the output image.

        Returns:
            npt.NDArray[np.uint8]: Transformed image.

        Raises:
            ValueError: If the image is not either grayscale or color.
        """
        # Check if image is either grayscale (2D) or color (3D)
        if len(image.shape) not in {2, 3}:
            raise ValueError("Image must be either grayscale or color.")

        # Apply perspective warp to the image using homography matrix and target resolution
        return cv2.warpPerspective(image, self.m, resolution_wh)